# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [ ]:
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

In [ ]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader

sys.path.insert(0, '/Users/ageller/VISUALIZATIONS/Firefly')
sys.path.insert(0,'/Users/agurvich/research/repos/Firefly')
from FireflyFlaskApp import spawnFireflyServer,killAllFireflyServers

## Start the server

In [ ]:
## spawnFireflyServer takes same *positional* arguments as startFireflyServer
process = spawnFireflyServer()

## Show Firefly in an IFrame

In [ ]:
url = "http://localhost:5000"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [ ]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("foo.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to foo.hdf5")

np.savetxt('foo.csv',coords)
print("output to foo.csv")

if not os.path.isdir('../static/data/foo_csv'):
    os.mkdir('../static/data/foo_csv')
    os.rename('foo.csv','../static/data/foo_csv/foo.csv')

## Send this data to the Flask app

In [ ]:
## test out the csv
fdir = '/Users/agurvich/research/repos/Firefly/static/data/foo_csv'
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_csv'

simple = SimpleReader("foo.csv",write_jsons_to_disk=False,extension='.csv')
#simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.csv')
simple.sendDataViaFlask()

In [ ]:
## test out the hdf5
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_hdf5'
#simple = SimpleReader("foo.hdf5",write_jsons_to_disk=False,extension='.hdf5')
simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.hdf5')
simple.sendDataViaFlask()

In [ ]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
print('sending to Firefly', sys.getsizeof(simple.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=simple.JSON)

In [ ]:
#test with FIRE data
fdir = '/Users/ageller/VISUALIZATIONS/FIREdata/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)
reader.sendDataViaFlask()

*To kill the Firefly server.*

In [ ]:
killAllFireflyServers(pid=process.pid)